In [1]:
import pandas as pd
import string
import numpy as np
import os
print(os.getcwd())

d:\OneDrive - MNSCU\Research\2023_AI_Text_Classification\AI_Generated_Text_Classfier


In [2]:
df1 = pd.read_csv('./data/features_GT.csv')
df1.sort_values(by='indexes', inplace=True)
print(df1.shape)

(3974, 50786)


In [3]:
df2 = pd.read_csv('./data/features_ChatGPT.csv')
df2.sort_values(by='indexes', inplace=True)
print(df2.shape)

(4557, 50786)


In [4]:
df1_filtered = df1.loc[df1['indexes'].isin(df2['indexes'])]
print(df1_filtered.shape)

(3643, 50786)


In [5]:
df2_filtered = df2.loc[df2['indexes'].isin(df1['indexes'])]
print(df2_filtered.shape)

(3643, 50786)


In [14]:
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

similarities = []
for i in tqdm(range(df1_filtered.shape[0])):
    try:
        gt_features = df1_filtered.iloc[i, 3:].to_numpy()
        #print(gt_features.shape)
        chatgpt_features = df2_filtered.iloc[i, 3:].to_numpy()
        #print(chatgpt_features.shape)
        similarity_score = cosine_similarity([gt_features], [chatgpt_features])
        #print('row ', i, ': score=', similarity_score)
        similarities.append(similarity_score)
    except Exception as e:
        similarity_score = 0
        similarities.append(similarity_score)
        print('Cosine error at row ', i, str(e))
        break
similarities = np.reshape(similarities, (-1,1))
similarities = pd.DataFrame(similarities, columns=['Cosine similarity'])
similarities.to_csv('./data/wiki_features_similarity.csv', index=False)

100%|██████████| 3643/3643 [02:01<00:00, 29.96it/s]
